In [ ]:
!pip install gradio

In [ ]:
import pandas as pd
import gradio as gr
from datetime import datetime
from datetime import date
from datetime import timedelta
#import sqlite3

In [ ]:
INF_URL = 'https://media.githubusercontent.com/media/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland/master/Aktuell_Deutschland_SarsCov2_Infektionen.csv'
VAC_URL = 'https://raw.githubusercontent.com/robert-koch-institut/COVID-19-Impfungen_in_Deutschland/master/Aktuell_Deutschland_Landkreise_COVID-19-Impfungen.csv'
GEO_URL = 'https://raw.githubusercontent.com/robert-koch-institut/SARS-CoV-2_Infektionen_in_Deutschland/master/2020-06-30_Deutschland_Landkreise_GeoDemo.csv'

inf_groupby_list = ['IdLandkreis','Meldedatum']
inf_filter = ['IdLandkreis','Meldedatum','AnzahlFall']
vac_groupby_list = ['LandkreisId_Impfort','Impfdatum']
vac_filter = ['LandkreisId_Impfort','Impfdatum','Anzahl']

infected = pd.read_csv(INF_URL)
vaccinated = pd.read_csv(VAC_URL)
geoData = pd.read_csv(GEO_URL)

inf_grouped = infected.groupby(inf_groupby_list, as_index=False,dropna=False).sum()
inf_grouped = inf_grouped[inf_filter]
vac_grouped = vaccinated.groupby(vac_groupby_list, as_index=False,dropna=False).sum()
vac_grouped = vac_grouped[vac_filter]

vac_grouped['LandkreisId_Impfort']=vac_grouped['LandkreisId_Impfort'].astype(str)
#inf_grouped['IdLandkreis']=inf_grouped['IdLandkreis'].astype(str)
inf_grouped['Meldedatum']=inf_grouped['Meldedatum'].astype(str)

def dropdownlist(url):
  df = pd.read_csv(url)
  dL = df['Gemeindename'].tolist()
  dL.sort()
  return dL

def filter_records(landeskreis, date):
    idLandKreis = geoData.loc[geoData['Gemeindename'] == landeskreis]['IdLandkreis'].values[0]  #type int
    data = {}
    data['Landeskreis'] = landeskreis
    data['Date'] = date
    data['Population'] = geoData.loc[geoData['IdLandkreis'] == idLandKreis]['EW_insgesamt'].values[0]
    if len(str(idLandKreis)) < 5:
      id = '0' + str(idLandKreis)
    else:
      id = str(idLandKreis)
    try:
      data['Vaccinated'] = vac_grouped.loc[(vac_grouped['Impfdatum'] == date) & (vac_grouped['LandkreisId_Impfort'] == id)]['Anzahl'].values[0]
    except:
      data['Vaccinated'] = "N/A"
    try:
      data['Infected'] = inf_grouped.loc[(inf_grouped['Meldedatum'] == date) & (inf_grouped['IdLandkreis'] == idLandKreis)]['AnzahlFall'].values[0]
    except:
      data['Infected'] = "N/A"
    #df = pd.DataFrame(columns=['Landeskreis','Date','Population','Vaccinated','Infected'])
    return pd.DataFrame([data])

iface = gr.Interface(filter_records,
  [
    gr.inputs.Dropdown(dropdownlist(GEO_URL)),
    gr.inputs.Textbox(default=str(date.today() - timedelta(days=1)))
  ],
  "dataframe",
  description="Enter Landeskreis and Date (YYYY-MM-DD).\nData taken from various sources in Robert Koch Institut repository: https://github.com/robert-koch-institut."
)

iface.launch()